# Writing out a USGSCSM ISD from a PDS3 LRO LROC NAC image

In [1]:
import os
os.environ["ALESPICEROOT"] = '/data/spice'

import json

import ale
from ale.drivers.mro_drivers import MroCtxPds3LabelNaifSpiceDriver
from ale.formatters.usgscsm_formatter import to_usgscsm
from ale.formatters.isis_formatter import to_isis

In [11]:
print(ale.spice_root)

/scratch/spice


## Instantiating an ALE driver

ALE drivers are objects that define how to acquire common ISD keys from an input image format, in this case we are reading in a PDS3 image using NAIF SPICE kernels for exterior orientation data. If the driver utilizes NAIF SPICE kernels, it is implemented as a [context manager](https://docs.python.org/3/reference/datamodel.html#context-managers) and will furnish metakernels when entering the context (i.e. when entering the `with` block) and free the metakernels on exit. This maintains the integrity of spicelib's internal data structures. These driver objects are short-lived and are input to a formatter function that consumes the API to create a serializable file format. `ale.formatters` contains available formatter functions. 

The default config file is located at `ale/config.yml` and is copied into your home directory at `.ale/config.yml` on first use of the library. The config file can be modified using a text editor. `ale.config` is loaded into memory as a dictionary. It is used to find metakernels for different missions. For example, there is an entry for MRO that points to `/usgs/cpkgs/isis3/data/mro/kernels/mk/` by default. If you want to use your own metakernels, you will need to udpate this path. For example, if the metakernels are located in `/data/ctxmro/mk/` the MRO entry should be updated with this path. If you are using the default metakernels, then you do not need to update the path.

ALE has a two step process for writing out an ISD: 1. Instantiate your driver (in this case `MroCtxPds3LabelNaifSpiceDriver`) within a context and 2. pass the driver object into a formatter (in this case, `to_usgscsm`).  

Requirements:
 * A PDS3 CTX MRO image
 * NAIF metakernels installed
 * Config file path for MRO (ale.config.mro) pointing to MRO NAIF metakernel directory 
 * A conda environment with ALE installed into it usisng the `conda install` command or created using the environment.yml file at the base of ALE.

In [6]:
%%time 
import pvl
# pvl.load("/usgs/cpkgs/isis3/data/mro/kernels/ck/kernels.0641.db")

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 9.06 µs


In [2]:
# import yaml

# with open("/scratch/spice/mro-m-spice-6-v1.0/mrosp_1000/extras/mk/mro_2012_v06.tm") as f:
#     text = f.read().encode()
#     d = yaml.load(text)


In [2]:
%%time
# change to desired PDS3 image path
file_name = '/home/acpaquette/F03_037064_1857_XN_05N218W.IMG'
from ale import util
import time 

class Spice(dict):
    def __getitem__(self, key):
        if key not in self.keys():
            self.__setitem__(key, util.duckpool(key))
        return super().__getitem__(key)
    
    def query(self, query):
        return util.query_kernel_pool(query)
        

# metakernels are furnsh-ed when entering the context (with block) with a driver instance
# most driver constructors simply accept an image path 
with MroCtxPds3LabelNaifSpiceDriver(file_name) as driver:
    # pass driver instance into formatter function
    pass
    usgscsmString = to_usgscsm(driver)
#     isisdict = to_isis(driver)

ValueError: Failed to find metakernels. mission: mro, year:2014, versions="latest" spice root = "/data/spice"

In [8]:
from collections import OrderedDict

dir(OrderedDict)

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'move_to_end',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [ ]:
import json

json.loads(json.dumps(isisdict, cls=ale.drivers.AleJsonEncoder))

In [8]:
dir(ale.drivers)

['ABC',
 'AleJsonEncoder',
 'IsisSpice',
 'OrderedDict',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__driver_modules__',
 '__file__',
 '__formatters__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'chain',
 'co_drivers',
 'date',
 'datetime',
 'dawn_drivers',
 'glob',
 'importlib',
 'inspect',
 'isis_ideal_drivers',
 'itertools',
 'json',
 'load',
 'loads',
 'lro_drivers',
 'mes_drivers',
 'mess_drivers',
 'mex_drivers',
 'mro_drivers',
 'np',
 'os',
 'pvl',
 'selene_drivers',
 'sort_drivers',
 'to_isis',
 'to_usgscsm',
 'traceback',
 'voyager_drivers',
 'zlib']

### Write ISD to disk 

ALE formatter functions generally return bytes or a string that can be written out to disk. ALE's USGSCSM formatter function returns a JSON encoded string that can be written out using any JSON library. 

USGSCSM requires the ISD to be colocated with the image file with a `.json` extension in place of the image extension.

In [4]:
# Load the json string into a dict
usgscsm_dict = json.loads(usgscsmString)

# Write the dict out to the associated file
json_file = os.path.splitext(file_name)[0] + '.json'

# Save off the json and read it back in to check if
# the json exists and was formatted correctly
with open(json_file, 'w') as fp:
    json.dump(usgscsm_dict, fp)
    
with open(json_file, 'r') as fp:
    usgscsm_dict = json.load(fp)
    
usgscsm_dict.keys()

dict_keys(['radii', 'sensor_position', 'sun_position', 'sensor_orientation', 'detector_sample_summing', 'detector_line_summing', 'focal_length_model', 'detector_center', 'starting_detector_line', 'starting_detector_sample', 'focal2pixel_lines', 'focal2pixel_samples', 'optical_distortion', 'image_lines', 'image_samples', 'name_platform', 'name_sensor', 'reference_height', 'name_model', 'interpolation_method', 'line_scan_rate', 'starting_ephemeris_time', 'center_ephemeris_time', 't0_ephemeris', 'dt_ephemeris', 't0_quaternion', 'dt_quaternion'])